In [27]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, LSTM
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import load_model
from tensorflow.keras.optimizers import Adam  # Make sure to use TensorFlow's Adam optimizer
from sklearn.model_selection import train_test_split
import re
import nltk

nltk.download("punkt")
nltk.download("stopwords")


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\ormec\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ormec\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [28]:
WPT = nltk.WordPunctTokenizer()
stopwordlist = nltk.corpus.stopwords.words("turkish")
stopwordlist

['acaba',
 'ama',
 'aslında',
 'az',
 'bazı',
 'belki',
 'biri',
 'birkaç',
 'birşey',
 'biz',
 'bu',
 'çok',
 'çünkü',
 'da',
 'daha',
 'de',
 'defa',
 'diye',
 'eğer',
 'en',
 'gibi',
 'hem',
 'hep',
 'hepsi',
 'her',
 'hiç',
 'için',
 'ile',
 'ise',
 'kez',
 'ki',
 'kim',
 'mı',
 'mu',
 'mü',
 'nasıl',
 'ne',
 'neden',
 'nerde',
 'nerede',
 'nereye',
 'niçin',
 'niye',
 'o',
 'sanki',
 'şey',
 'siz',
 'şu',
 'tüm',
 've',
 'veya',
 'ya',
 'yani']

In [29]:
import random as r
dataset = pd.read_excel("C:/Users/ormec/DLearning/Datasets/TurkishTweets.xlsx")
dataset

,Tweet,Etiket
0,Allah'ım çıldıracağım. Yemin ederim çıldıracağ...,kızgın
1,site ne zaman çalıştıda ürün stokları bitti di...,kızgın
2,Sebebi neydi ki diye bağıracağım şimdi az kaldı,kızgın
3,Black friday ne amk bu yıl çıkardınız bu bokud...,kızgın
4,kısa kısa cevaplar vericeksen niye yazıyorsun ...,kızgın
...,...,...
3995,Aynı şeyleri yapmaktan sıkıldım,üzgün
3996,"Olacağı varsa oluyor ya hani, biteceği varsa d...",üzgün
3997,Çok şey istiyorum ama sonu hep fiyasko MUTSUZUM,üzgün
3998,Garip bir şekilde canım çok yanıyor moralim bozuk,üzgün


In [30]:
dataset.Tweet = dataset.Tweet.apply(str)

In [31]:
#Noktalamaların temizlenmesi 
dataset["Tweet"] = dataset["Tweet"].apply(lambda x:re.sub('[,\.!?:()"]','',x))

#Büyük harf küçük harfe
dataset["Tweet"] = dataset["Tweet"].apply(lambda x:x.lower())

#Boşlukların temizlenmesi
dataset["Tweet"] = dataset["Tweet"].apply(lambda x:x.strip())

#Etkisiz kelimelerin kaldırılması
def token(values):
  words = nltk.tokenize.word_tokenize(values)
  stop_words = nltk.corpus.stopwords.words('turkish')
  filtered_words = [word for word in words if word not in stop_words]
  not_stopword_doc = "".join(filtered_words)
  return not_stopword_doc

dataset["Tweet"] = dataset["Tweet"].apply(lambda x: token(x))
dataset

,Tweet,Etiket
0,allah'ımçıldıracağımyeminederimçıldıracağımsin...,kızgın
1,sitezamançalıştıdaürünstoklarıbittidiyormalmıs...,kızgın
2,sebebineydibağıracağımşimdikaldı,kızgın
3,blackfridayamkyılçıkardınızbokudamalmısınıznes...,kızgın
4,kısakısacevaplarvericeksenyazıyorsungerizekalı,kızgın
...,...,...
3995,aynışeyleriyapmaktansıkıldım,üzgün
3996,olacağıvarsaoluyorhanibiteceğivarsabitsinyoruldum,üzgün
3997,istiyorumsonufiyaskomutsuzum,üzgün
3998,garipbirşekildecanımyanıyormoralimbozuk,üzgün


In [32]:
tweet = dataset["Tweet"].values.tolist()
sentiment = dataset["Etiket"].values.tolist()
print(sentiment)

for n,i in enumerate(sentiment):
    if i=="kızgın":
        sentiment[n]=0
    if i=="korku":
        sentiment[n]=1
    if i=="mutlu":
        sentiment[n]=2
    if i=="surpriz":
        sentiment[n]=3
    if i=="üzgün":
        sentiment[n]=4
    
X_train,X_test,y_train,y_test = train_test_split(tweet,sentiment,test_size=0.2,random_state=42)

['kızgın', 'kızgın', 'kızgın', 'kızgın', 'kızgın', 'kızgın', 'kızgın', 'kızgın', 'kızgın', 'kızgın', 'kızgın', 'kızgın', 'kızgın', 'kızgın', 'kızgın', 'kızgın', 'kızgın', 'kızgın', 'kızgın', 'kızgın', 'kızgın', 'kızgın', 'kızgın', 'kızgın', 'kızgın', 'kızgın', 'kızgın', 'kızgın', 'kızgın', 'kızgın', 'kızgın', 'kızgın', 'kızgın', 'kızgın', 'kızgın', 'kızgın', 'kızgın', 'kızgın', 'kızgın', 'kızgın', 'kızgın', 'kızgın', 'kızgın', 'kızgın', 'kızgın', 'kızgın', 'kızgın', 'kızgın', 'kızgın', 'kızgın', 'kızgın', 'kızgın', 'kızgın', 'kızgın', 'kızgın', 'kızgın', 'kızgın', 'kızgın', 'kızgın', 'kızgın', 'kızgın', 'kızgın', 'kızgın', 'kızgın', 'kızgın', 'kızgın', 'kızgın', 'kızgın', 'kızgın', 'kızgın', 'kızgın', 'kızgın', 'kızgın', 'kızgın', 'kızgın', 'kızgın', 'kızgın', 'kızgın', 'kızgın', 'kızgın', 'kızgın', 'kızgın', 'kızgın', 'kızgın', 'kızgın', 'kızgın', 'kızgın', 'kızgın', 'kızgın', 'kızgın', 'kızgın', 'kızgın', 'kızgın', 'kızgın', 'kızgın', 'kızgın', 'kızgın', 'kızgın', 'kızgın', 'kızgın',

In [33]:
dataset

,Tweet,Etiket
0,allah'ımçıldıracağımyeminederimçıldıracağımsin...,kızgın
1,sitezamançalıştıdaürünstoklarıbittidiyormalmıs...,kızgın
2,sebebineydibağıracağımşimdikaldı,kızgın
3,blackfridayamkyılçıkardınızbokudamalmısınıznes...,kızgın
4,kısakısacevaplarvericeksenyazıyorsungerizekalı,kızgın
...,...,...
3995,aynışeyleriyapmaktansıkıldım,üzgün
3996,olacağıvarsaoluyorhanibiteceğivarsabitsinyoruldum,üzgün
3997,istiyorumsonufiyaskomutsuzum,üzgün
3998,garipbirşekildecanımyanıyormoralimbozuk,üzgün


In [34]:
from tensorflow.keras.preprocessing.text import Tokenizer

tokenizer = Tokenizer(num_words = 10000) #Metin parçasını böler.
tokenizer.fit_on_texts(tweet)
tokenizer.word_index

{'benkendimlesavaşırkenseninkazanmanbanabirazsürprizoldu': 1,
 'kendimikarşımaalıpkonuşmakistiyorumbakgerizekalıişböyleolmaz': 2,
 'i̇çinizdeöldürdüğünüzkişiyikarşınızdagörmekkorkutucudeğilmi': 3,
 'şaşkınlıkvericişampiyonlarliginderakiplerineköksöktürentakımınkadarkötüoynamasınışaşkınlıklaizliyorum': 4,
 'hayatınespirianlayışışaşkınlıkverici': 5,
 'birinizyüzündenhepinizeduvarlarördümüzgünüm': 6,
 "allah'ımetrafbeyinsizkaynıyorçıldıracağım": 7,
 'allahkimseyigerizekalıinsanlarlasınamasın': 8,
 'nefretimizelektriğedönüştürülebilseydibütündünyayıaydınlatmayayeterdi': 9,
 'bomboşsaçmabugünöfkeliyim': 10,
 'meselabenkızdığımıaçıklamayaçalışırkensinirleniyorum': 11,
 'kendikendimekonuşurkenbaşkasıarayagirincesinirleniyorum': 12,
 'süreklisinirleniyorumartıkakışınabıraktımneyesinirlendiğimifalanbilmiyorum': 13,
 'domuzetiyersinsensinirdenküplerebindimuyuyamıyorum': 14,
 'fenaiftiraatıldıüstümeansinirliyim': 15,
 'zekiinsanlarçabuksinirlenirokudumbiryerdeoldubilmiyorumbirandaküplerebindim': 

In [35]:
X_train_tokens = tokenizer.texts_to_sequences(X_train)
X_test_tokens = tokenizer.texts_to_sequences(X_test)
#Metin dizisi sayılara dönüşür . Bugün hava çok güzel cümlesi 0123 olarak ifade edilir.
num_tokens = [len(tokens) for tokens in X_train_tokens + X_test_tokens]
num_tokens = np.array(num_tokens)

max_tokens = np.mean(num_tokens) + 2*np.std(num_tokens) # Her cümle farklı uzunluktadır , ortalama bulunur ve her cümle
max_tokens = int(max_tokens)                            # ortalama uzaklığa getirilir.Aşağıda kodu görebilirsin.

num_tokens,max_tokens

(array([1, 1, 1, ..., 1, 1, 1]), 1)

In [36]:
X_train_padding = pad_sequences(X_train_tokens,maxlen=max_tokens)
X_test_padding = pad_sequences(X_test_tokens,maxlen=max_tokens)

X_test_padding[15]

array([3460])

In [37]:
idx = tokenizer.word_index
inverse_map = dict(zip(idx.values(),idx.keys()))
def tokens_to_string(tokens):
    words = [inverse_map[token] for token in tokens in token!=0]
    text = ' '.join(words)
    return text             # Token halindeki veriler tekrar cümleye çevrilir.

In [38]:
import tensorflow as tf

model = Sequential()
embedding_dim = 100
model.add(Embedding(input_dim=10000,
                    output_dim=embedding_dim,
                    input_length=max_tokens,
                    name="embedding_layer"
                    ))

model.add(LSTM(units=32,return_sequences=True,dropout=0.4,recurrent_dropout=0.2))
model.add(LSTM(units=16,return_sequences=True,dropout=0.4,recurrent_dropout=0.2))
model.add(LSTM(units=8,dropout=0.4,recurrent_dropout=0.4))
model.add(Dense(5, activation='softmax'))

optimizer = tf.keras.optimizers.Adam(learning_rate=1e-3)

model.compile(loss='sparse_categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])


In [40]:
y_train = np.array(y_train)
history=model.fit(X_train_padding,y_train,validation_split=0.25,epochs=10,batch_size=8)

Epoch 1/10
300/300 [==============================] - 9s 20ms/step - loss: 1.6097 - accuracy: 0.1996 - val_loss: 1.6098 - val_accuracy: 0.1850
Epoch 2/10
300/300 [==============================] - 5s 15ms/step - loss: 1.6072 - accuracy: 0.2125 - val_loss: 1.6086 - val_accuracy: 0.2050
Epoch 3/10
300/300 [==============================] - 5s 15ms/step - loss: 1.0625 - accuracy: 0.6175 - val_loss: 1.8980 - val_accuracy: 0.2250
Epoch 4/10
300/300 [==============================] - 5s 16ms/step - loss: 0.5732 - accuracy: 0.7812 - val_loss: 2.1696 - val_accuracy: 0.2450
Epoch 5/10
300/300 [==============================] - 4s 14ms/step - loss: 0.3947 - accuracy: 0.8692 - val_loss: 2.5143 - val_accuracy: 0.2412
Epoch 6/10
300/300 [==============================] - 4s 13ms/step - loss: 0.2526 - accuracy: 0.9375 - val_loss: 2.7712 - val_accuracy: 0.2262
Epoch 7/10
300/300 [==============================] - 4s 14ms/step - loss: 0.1682 - accuracy: 0.9621 - val_loss: 3.0827 - val_accuracy: 0.2250

In [41]:
y_test = np.array(y_test)
result = model.evaluate(X_test_padding,y_test)

25/25 [==============================] - 1s 1ms/step - loss: 4.0192 - accuracy: 0.2087


In [42]:
model.save(f'../LSTM_Saved')

INFO:tensorflow:Assets written to: ../LSTM_Saved\assets


INFO:tensorflow:Assets written to: ../LSTM_Saved\assets
